# Import Event Log

In [1]:
import pandas as pd
import numpy as np
import pm4py
from pm4py.objects.conversion.log import converter as log_converter

if __name__ == "__main__":
    # Read the CSV file
    dataframe_log = pd.read_csv('../../data/logs/gigantic_log.csv', sep=',')

    # Format the dataframe
    dataframe_log = pm4py.format_dataframe(
        dataframe_log,
        case_id='case:concept:name',
        activity_key='concept:name',
        timestamp_key='time:timestamp'
    )

    # Convert the dataframe to event log
    log = log_converter.apply(dataframe_log)

/var/folders/s_/ch_w_j2d0sqf6dbdc0_224m40000gq/T/ipykernel_3433/1982556689.py:11: DeprecatedWarning: format_dataframe is deprecated as of 2.3.0 and will be removed in 3.0.0. the format_dataframe function does not need application anymore.
  dataframe_log = pm4py.format_dataframe(


In [2]:
dataframe_log

,concept:name,timestamp,time:timestamp,case:label,case:concept:name,org:resource,@@index,@@case_index
0,Random activity 47,NaN,2010-01-01 00:00:00+00:00,Insert,1,Random user 44,0,0
1,Activity A,NaN,2010-01-01 00:01:00+00:00,Insert,1,Micheline,1,0
2,Activity T,NaN,2010-01-01 00:02:00+00:00,Insert,1,Steve,2,0
3,Activity AC,NaN,2010-01-01 00:04:00+00:00,Insert,1,Ling,3,0
4,Activity AE,NaN,2010-01-01 00:05:00+00:00,Insert,1,Amanda,4,0
...,...,...,...,...,...,...,...,...
29824,Activity P,NaN,2010-06-01 22:58:00+00:00,normal,999,Hoyt,29824,4999
29825,Activity M,NaN,2010-06-01 22:59:00+00:00,normal,999,Maryellen,29825,4999
29826,Activity D,NaN,2010-06-01 23:00:00+00:00,normal,999,Shandi,29826,4999
29827,Activity S,NaN,2010-06-01 23:02:00+00:00,normal,999,Chan,29827,4999


# Drop unnessary columns

In [3]:
dataframe_log = dataframe_log.drop(columns=['timestamp'])

In [4]:
dataframe_log = dataframe_log.drop(columns=['case:concept:name'])

In [5]:
dataframe_log = dataframe_log.drop(columns=['@@index'])

In [6]:
dataframe_log = dataframe_log.drop(columns=['time:timestamp'])

In [7]:
dataframe_log

,concept:name,case:label,org:resource,@@case_index
0,Random activity 47,Insert,Random user 44,0
1,Activity A,Insert,Micheline,0
2,Activity T,Insert,Steve,0
3,Activity AC,Insert,Ling,0
4,Activity AE,Insert,Amanda,0
...,...,...,...,...
29824,Activity P,normal,Hoyt,4999
29825,Activity M,normal,Maryellen,4999
29826,Activity D,normal,Shandi,4999
29827,Activity S,normal,Chan,4999


# Insert Start & End markers

In [8]:
# Define a function to insert start and end markers
def add_markers(df):
    # Identify unique case indices
    case_indices = df['@@case_index'].unique()
    
    # Prepare a container for new DataFrame rows
    new_rows = []
    
    # Iterate over each case index to add start and end markers
    for case_index in case_indices:
        # Create a start marker row with all columns except @@case_index set to 'Start'
        start_row = {col: 'Start' if col != '@@case_index' else case_index for col in df.columns}
        
        # Create an end marker row with all columns except @@case_index set to 'End'
        end_row = {col: 'End' if col != '@@case_index' else case_index for col in df.columns}
        
        # Append start row, rows for the current case, and end row
        new_rows.append(start_row)
        new_rows.extend(df[df['@@case_index'] == case_index].to_dict('records'))
        new_rows.append(end_row)
    
    # Convert the list of rows into a DataFrame
    return pd.DataFrame(new_rows)

# Apply the function to add start and end markers to the dataframe
modified_dataframe = add_markers(dataframe_log)

# Preprocess

In [9]:
modified_dataframe

,concept:name,case:label,org:resource,@@case_index
0,Start,Start,Start,0
1,Random activity 47,Insert,Random user 44,0
2,Activity A,Insert,Micheline,0
3,Activity T,Insert,Steve,0
4,Activity AC,Insert,Ling,0
...,...,...,...,...
39824,Activity M,normal,Maryellen,4999
39825,Activity D,normal,Shandi,4999
39826,Activity S,normal,Chan,4999
39827,Activity B,normal,Ronald,4999


In [10]:
codes, uniques = pd.factorize(modified_dataframe['org:resource'])
modified_dataframe['org:resource'] = codes

In [11]:
codes, uniques = pd.factorize(modified_dataframe['concept:name'])
modified_dataframe['concept:name'] = codes

### Padding for Cases with less then 5 events

In [12]:
# Calculating the frequency of each unique value in '@@case_index'
frequency = modified_dataframe['@@case_index'].value_counts()

# Finding the minimum occurrence
min_occurrence = frequency.min()

min_occurrence

3

In [13]:
# Assuming your dataframe might have different types, let's create a generic function to add rows
def add_rows(group, num_rows, case_index_value):
    # For each column, determine the appropriate "zero" value (int 0, string '', etc.)
    additional_rows = pd.DataFrame({
        column: 0 if pd.api.types.is_numeric_dtype(group[column]) else '' for column in group.columns
    }, index=range(num_rows))
    
    # Set the @@case_index column to the current case index value
    additional_rows['@@case_index'] = case_index_value
    
    # Append the additional rows to the group
    return pd.concat([group, additional_rows], ignore_index=True)

# Function to pad cases with less than 5 events
def pad_cases(df):
    # Group by @@case_index
    groups = df.groupby('@@case_index')
    
    # Placeholder for modified groups
    modified_groups = []
    
    for name, group in groups:
        # Calculate the number of events to add
        events_to_add = 5 - len(group)
        
        if events_to_add > 0:
            # Add the required number of rows
            group = add_rows(group, events_to_add, name)
        
        # Append the modified group to the list
        modified_groups.append(group)
    
    # Concatenate all modified groups back into a single DataFrame
    return pd.concat(modified_groups, ignore_index=True)

In [14]:
# Apply the padding function
modified_dataframe = pad_cases(modified_dataframe)
modified_dataframe

,concept:name,case:label,org:resource,@@case_index
0,0,Start,0,0
1,1,Insert,1,0
2,2,Insert,2,0
3,3,Insert,3,0
4,4,Insert,4,0
...,...,...,...,...
40043,13,normal,18,4999
40044,14,normal,19,4999
40045,15,normal,20,4999
40046,8,normal,17,4999


# Generate sliding windows

In [15]:
df_resource = modified_dataframe[['org:resource', '@@case_index']]
df_activity = modified_dataframe[['concept:name', '@@case_index']]

In [16]:
def generate_sliding_windows(df, case_id_column='@@case_index', window_size=5):
    windows = []
    targets = []
    case_indices = []

    # Iterate over each unique case
    for case_id in df[case_id_column].unique():
        # Extract the case
        case_data = df[df[case_id_column] == case_id]
        
        # Convert case_data to a NumPy array and drop the case_id_column
        case_data_array = case_data.drop(columns=[case_id_column]).to_numpy()

        # Adjusting the condition to correctly reflect window_size without needing an additional +1
        # Now it correctly considers window_size as including the target event
        if len(case_data_array) >= window_size:
            # Adjust the loop to generate sliding windows of size window_size - 1 for the inputs and use the next event as the target
            for i in range(len(case_data_array) - window_size + 1):
                # window now has window_size - 1 events
                window = case_data_array[i:i + window_size - 1]
                # The target is the event immediately following the window
                target = case_data_array[i + window_size - 1]
                windows.append(window)
                targets.append(target)
                case_indices.append(case_id)  # Store the case_id corresponding to the window

    # Convert lists to numpy arrays for easier handling and to ensure they are two-dimensional
    windows_array = np.array(windows)
    targets_array = np.array(targets)
    case_indices_array = np.array(case_indices)
    
    return windows_array, targets_array, case_indices_array

In [17]:
windows_resource, targets_resource, case_indices = generate_sliding_windows(df_resource)
windows_activity, targets_activity, case_indices = generate_sliding_windows(df_activity)

# LSTM

### Architecture

In [18]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Concatenate, Embedding

# Assuming these values as placeholders, replace them with actual counts from your data
num_resources = modified_dataframe['org:resource'].nunique()
num_activities = modified_dataframe['concept:name'].nunique()

embedding_dim_resource = 50
embedding_dim_activity = 50

time_steps = 4

# Input layers
input_resource = Input(shape=(time_steps,), name='input_resource')
input_activity = Input(shape=(time_steps,), name='input_activity')

# Embedding layers
embedding_resource = Embedding(input_dim=num_resources, output_dim=embedding_dim_resource, input_length=time_steps)(input_resource)
embedding_activity = Embedding(input_dim=num_activities, output_dim=embedding_dim_activity, input_length=time_steps)(input_activity)

# LSTM layers
lstm_resource = LSTM(25, return_sequences=False)(embedding_resource)
lstm_activity = LSTM(25, return_sequences=False)(embedding_activity)

# Concatenate outputs
concatenated = Concatenate(axis=-1)([lstm_resource, lstm_activity])

# Output layers
output_resource = Dense(num_resources, activation='softmax', name='output_resource')(concatenated)
output_activity = Dense(num_activities, activation='softmax', name='output_activity')(concatenated)

# Create and compile the model
model = Model(inputs=[input_resource, input_activity], 
              outputs=[output_resource, output_activity])

model.compile(optimizer='adam', 
              loss={'output_resource': 'categorical_crossentropy', 
                    'output_activity': 'categorical_crossentropy'},
              metrics={'output_resource': 'accuracy', 
                       'output_activity': 'accuracy'})

model.summary()


2024-06-08 16:00:53.470470: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_resource (InputLayer  [(None, 4)]                  0         []                            
 )                                                                                                
                                                                                                  
 input_activity (InputLayer  [(None, 4)]                  0         []                            
 )                                                                                                
                                                                                                  
 embedding (Embedding)       (None, 4, 50)                7000      ['input_resource[0][0]']      
                                                                                              

# Data Splitting

In [19]:
from sklearn.model_selection import train_test_split

# Split the resource data
train_resource, test_resource, train_targets_resource, test_targets_resource = train_test_split(
    windows_resource, targets_resource, test_size=0.3, random_state=42)

# Split the activity data
train_activity, test_activity, train_targets_activity, test_targets_activity = train_test_split(
    windows_activity, targets_activity, test_size=0.3, random_state=42)


# Training

In [20]:
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import LearningRateScheduler

# EarlyStopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

def cyclic_lr(epoch, lr):
    # Example function that modulates LR within a range for each epoch
    # Customize this function based on your cyclic learning rate policy
    max_lr = 0.01  # Maximum LR
    base_lr = 0.001  # Base LR
    step_size = 10  # Number of epochs for half a cycle
    cycle = np.floor(1 + epoch / (2 * step_size))
    x = np.abs(epoch / step_size - 2 * cycle + 1)
    lr = base_lr + (max_lr - base_lr) * np.maximum(0, (1 - x))
    return lr

lr_scheduler = LearningRateScheduler(cyclic_lr)

In [21]:
import numpy as np
from tensorflow.keras.utils import to_categorical


train_targets_resource_cat = to_categorical(train_targets_resource, num_classes=num_resources)
test_targets_resource_cat = to_categorical(test_targets_resource, num_classes=num_resources)

train_targets_activity_cat = to_categorical(train_targets_activity, num_classes=num_activities)
test_targets_activity_cat = to_categorical(test_targets_activity, num_classes=num_activities)


# Note: Assuming timestamp targets are continuous and don't need to be converted to categorical

history = model.fit([train_resource, train_activity], 
                    [train_targets_resource_cat, train_targets_activity_cat],
                    epochs=25,
                    batch_size=64,
                    validation_data=([test_resource, test_activity], 
                                     [test_targets_resource_cat, test_targets_activity_cat]),
                    verbose=1,
                    callbacks=[lr_scheduler])  # Add other callbacks as needed

Epoch 1/25
220/220 [==============================] - 7s 10ms/step - loss: 6.8664 - output_resource_loss: 3.6562 - output_activity_loss: 3.2101 - output_resource_accuracy: 0.2511 - output_activity_accuracy: 0.2811 - val_loss: 5.2574 - val_output_resource_loss: 3.0444 - val_output_activity_loss: 2.2130 - val_output_resource_accuracy: 0.2999 - val_output_activity_accuracy: 0.4454 - lr: 0.0010
Epoch 2/25
220/220 [==============================] - 1s 5ms/step - loss: 4.0248 - output_resource_loss: 2.5893 - output_activity_loss: 1.4355 - output_resource_accuracy: 0.3926 - output_activity_accuracy: 0.6758 - val_loss: 3.2775 - val_output_resource_loss: 2.2988 - val_output_activity_loss: 0.9787 - val_output_resource_accuracy: 0.4680 - val_output_activity_accuracy: 0.8100 - lr: 0.0019
Epoch 3/25
220/220 [==============================] - 2s 7ms/step - loss: 2.7705 - output_resource_loss: 2.0350 - output_activity_loss: 0.7355 - output_resource_accuracy: 0.5045 - output_activity_accuracy: 0.8444 

In [22]:
evaluation = model.evaluate(
    [test_resource, test_activity],
    [test_targets_resource_cat, test_targets_activity_cat]
)

print(evaluation)

188/188 [==============================] - 2s 9ms/step - loss: 1.9379 - output_resource_loss: 1.5031 - output_activity_loss: 0.4349 - output_resource_accuracy: 0.6165 - output_activity_accuracy: 0.9270
[1.9379217624664307, 1.5030626058578491, 0.4348585605621338, 0.6164588332176208, 0.9270157814025879]


In [31]:
# Save the model to an H5 file
model.save('dapnn_gigantic.h5')

/opt/anaconda3/lib/python3.9/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


# Anomaly Score Computation

In [26]:
# Generate predictions for all inputs
predictions = model.predict([windows_resource, windows_activity])

# Extract predictions for categorical attributes (softmax probabilities)
predictions_resource = predictions[0]        # ID predictions
predictions_activity = predictions[1]  # Resource predictions

627/627 [==============================] - 2s 2ms/step


In [27]:
import numpy as np
from tensorflow.keras.utils import to_categorical

def compute_anomaly_scores(predictions, actuals):
    # For categorical predictions, convert actuals to one-hot for comparison
    actuals_one_hot = to_categorical(actuals, num_classes=predictions.shape[-1])
    
    max_predictions = np.max(predictions, axis=-1)
    actual_predictions = np.sum(predictions * actuals_one_hot, axis=-1)  # Extract the probability of the actual class
    
    anomaly_scores = (max_predictions - actual_predictions) / max_predictions
    
    return anomaly_scores

# Assuming targets_id, targets_resource, targets_activity, targets_role are the true values for these attributes
anomaly_scores_resource = compute_anomaly_scores(predictions_resource, targets_resource)
anomaly_scores_activity = compute_anomaly_scores(predictions_activity, targets_activity)

In [29]:
import numpy as np

def classify_cases(anomaly_scores_resource, anomaly_scores_activity, threshold=0.98):
    # Ensure all inputs are numpy arrays of the same shape
    anomaly_scores_resource = np.array(anomaly_scores_resource).flatten()
    anomaly_scores_activity = np.array(anomaly_scores_activity).flatten()

    # Check if all arrays have the same length
    if not (len(anomaly_scores_resource) == len(anomaly_scores_activity)):
        raise ValueError("All input anomaly scores must have the same length.")

    # Find the maximum anomaly score across all attributes for each case
    max_scores = np.maximum.reduce([anomaly_scores_resource, anomaly_scores_activity])

    # Classify cases as anomalous if the maximum anomaly score exceeds the threshold
    anomalous_cases = max_scores > threshold
    
    return anomalous_cases

# Now use the anomaly scores for resource, activity, and timestamp in the classification
anomalous_cases = classify_cases(anomaly_scores_resource, anomaly_scores_activity)

In [30]:
anomalous_cases

array([ True, False, False, ..., False, False, False])

# Mapping

### True: anomaly, False: no anomaly

In [31]:
import pandas as pd

# Create a DataFrame from the case_indices_array corresponding to case_resource
mapping = pd.DataFrame({'case': case_indices})
mapping['predicted'] = anomalous_cases
mapping

,case,predicted
0,0,True
1,0,False
2,0,False
3,0,False
4,0,False
...,...,...
20043,4999,False
20044,4999,False
20045,4999,False
20046,4999,False


In [32]:
case_prediction = mapping.groupby('case')['predicted'].any()
case_prediction


case
0        True
1       False
2       False
3       False
4       False
        ...  
4995     True
4996    False
4997    False
4998    False
4999    False
Name: predicted, Length: 5000, dtype: bool

# Ground Truth

In [33]:
import pm4py
from pm4py.objects.log.importer.xes import importer as xes_importer
from pm4py.objects.bpmn.importer import importer as bpmn_importer
from pm4py.algo.conformance.alignments.petri_net import algorithm as alignments_petri

# 2. Import the given BPMN model
bpmn_graph = bpmn_importer.apply("../../data/model/gigantic.bpmn")

# 3. Convert the BPMN to a Petri net
net, im, fm = pm4py.convert_to_petri_net(bpmn_graph)

# 4. Perform alignment-based conformance checking
alignments = alignments_petri.apply(log, net, im, fm)

# Calculate and print diagnostics
fit_traces = sum(1 for trace in alignments if trace['fitness'] == 1.0)

print(f"Total traces: {len(log)}")
print(f"Conform traces: {fit_traces}")
print(f"Non-Conform traces: {len(log) - fit_traces}")

aligning log, completed variants ::   0%|          | 0/738 [00:00<?, ?it/s]

Total traces: 5000
Conform traces: 3747
Non-Conform traces: 1253


In [34]:
conformity_array = [int(trace['fitness']) for trace in alignments]

ground_truth = pd.DataFrame({'conformity': conformity_array})
ground_truth['predicted'] = case_prediction

# Convert False to 0 and True to 1
ground_truth['predicted'] = [int(value) for value in ground_truth['predicted']]
ground_truth['predicted'] = 1 - ground_truth['predicted']
ground_truth

,conformity,predicted
0,0,0
1,1,1
2,1,1
3,1,1
4,1,1
...,...,...
4995,0,0
4996,0,1
4997,1,1
4998,1,1


# Evaluation

In [35]:
# Calculating TP, TN, FP, FN
TP = ((ground_truth['conformity'] == 1) & (ground_truth['predicted'] == 1)).sum()
TN = ((ground_truth['conformity'] == 0) & (ground_truth['predicted'] == 0)).sum()
FP = ((ground_truth['conformity'] == 0) & (ground_truth['predicted'] == 1)).sum()
FN = ((ground_truth['conformity'] == 1) & (ground_truth['predicted'] == 0)).sum()

In [36]:
# Calculate accuracy
accuracy = (TP + TN) / (TP + TN + FP + FN)
print(f"Accuracy: {accuracy:.3f}")

Accuracy: 0.796


In [37]:
# Calculate f1

precision = TP / (TP + FP)
recall = TP / (TP + FN)

f1 = 2 * ((precision * recall) / (precision + recall))
print(f"F1: {f1:.3f}")

F1: 0.874


### Dev (Non Conform Traces)

In [38]:
# Calculate precision for Dev
precision = TN / (TN + FN)
print(f"Precision: {precision:.3f}")

Precision: 0.675


In [39]:
# Calculate recall for Dev
recall = TN / (TN + FP)
print(f"Recall: {recall:.3f}")

Recall: 0.362


### No Dev (Conform Traces)

In [40]:
# Calculate precision for No Dev
precision = TP / (TP + FP)
print(f"Precision: {precision:.3f}")

Precision: 0.815


In [41]:
# Calculate recall for No Dev
recall = TP / (TP + FN)
print(f"Recall: {recall:.3f}")

Recall: 0.942


### AUC

In [42]:
import pandas as pd
from sklearn.metrics import roc_auc_score

# Assuming ground_truth is your DataFrame
# Make sure 'conformity' contains actual labels (0 or 1)
# and 'predicted' contains predicted probabilities or scores
auc_roc = roc_auc_score(ground_truth['conformity'], ground_truth['predicted'])
auc_roc

0.6516762225955279